## Graph Dynamical Systems

This notebooks contains the experiments to evaluate graph edit networks on simple graph dynamical systems, namely the edit cycles, degree rules, and game of life datasets.

### Hyperparameter setup

In [1]:
import time
import numpy as np
import torch
import pytorch_graph_edit_networks as gen
from torch_geometric.utils import dense_to_sparse
import baseline_models_with_vgrnn as baseline_models
import os
import time
import hep_th
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
# torch.cuda.set_device(0)


# model hyperparameters
num_layers = 2
dim_hid = 64

# training hyperparameters
learning_rate  = 1E-3
weight_decay   = 1E-5
loss_threshold = 1E-3
#TODO run on more
max_epochs     = 3000

print_step     = 100

R = 5        # number of repetitions for each experiment
N_test = 10  # number of test time series we use to evaluate learning afterwards

In [2]:
# if torch.cuda.is_available():
#     dev = "cuda:0"
# else:
#     dev = "cpu"
#
# dev

## Model setup

In [3]:
# SETUP FUNCTIONS
def setup_vgae(dim_in, nonlin):
    return baseline_models.VGAE(num_layers=num_layers,
                                dim_in=dim_in,
                                dim_hid=dim_hid,
                                beta=1E-3,
                                sigma_scaling=1E-3,
                                nonlin=nonlin)


def setup_vgrnn(dim_in, nonlin):
    return baseline_models.VGRNN(num_layers=num_layers,
                                 dim_in=dim_in,
                                 dim_hid=dim_hid)


def setup_gen(dim_in, nonlin):
    return gen.GEN(num_layers=num_layers,
                   dim_in=dim_in,
                   dim_hid=dim_hid,
                   nonlin=nonlin)

In [4]:
# LOSS FUNCTIONS
loss_fun = gen.GEN_loss()
crossent_loss_fun = gen.GEN_loss_crossent()
def vgae_loss(model, A, X, delta, Epsilon, state=None):
    model = model
    B = A + Epsilon
    # delete all outgoing and incoming edges of deleted nodes
    B[delta < -0.5, :] = 0
    B[:, delta < -0.5] = 0
    loss = model.compute_loss(torch.tensor(A, dtype=torch.float),
                              torch.tensor(B, dtype=torch.float),
                              torch.tensor(X, dtype=torch.float))

    return loss, state


def vgrnn_loss(model, A, X, delta, Epsilon, state=None):
    model = model
    A = torch.tensor(A, dtype=torch.float)
    A = A
    edge_index, _ = dense_to_sparse(A)
    edge_index = edge_index
    X = torch.Tensor(X)
    X = X
    predicted = model(X, edge_index, hidden_in=state)
    predicted = predicted
    predicted, state = predicted[:-1], predicted[-1]
    state=state

    target = A + Epsilon
    target = target
    target[delta < -0.5, :] = 0
    target[:, delta < -0.5] = 0
    # print('------')
    return model.compute_loss(*predicted, target), state


def gen_loss_crossent(model, A, X, delta, Epsilon, state=None):
    delta_pred, Epsilon_pred = model(torch.tensor(A, dtype=torch.float),
                                     torch.tensor(X, dtype=torch.float))
    loss = crossent_loss_fun(delta_pred, Epsilon_pred,
                             torch.tensor(delta, dtype=torch.float),
                             torch.tensor(Epsilon, dtype=torch.float),
                             torch.tensor(A, dtype=torch.float))

    return loss, state


def gen_loss(model, A, X, delta, Epsilon, state=None):
    delta_pred, Epsilon_pred = model(torch.tensor(A, dtype=torch.float),
                                     torch.tensor(X, dtype=torch.float))
    loss = loss_fun(delta_pred, Epsilon_pred,
                    torch.tensor(delta, dtype=torch.float),
                    torch.tensor(Epsilon, dtype=torch.float),
                    torch.tensor(A, dtype=torch.float))

    return loss, state

In [5]:
# PREDICTION FUNCTIONS
def vgae_pred(model, A, X, state=None):
    B = model(torch.tensor(A, dtype=torch.float), torch.tensor(X, dtype=torch.float))
    B = B.detach().numpy()
    Epsilon = B - A
    delta = np.zeros(A.shape[0])
    delta[np.sum(B, 1) < 0.5] = -1.
    Epsilon[delta < -0.5, :] = 0.
    Epsilon[:, delta < -0.5] = 0.
    return delta, Epsilon, state


def vgrnn_pred(model, A, X, state=None):
    model = model
    A = torch.tensor(A, dtype=torch.float)
    A = A
    edge_index = dense_to_sparse(A)
    edge_index = edge_index
    predicted = model(torch.tensor(X), edge_index, hidden_in=state)
    predicted = predicted
    predicted, state = predicted[:-1], predicted[-1]
    predicted = predicted
    state = state

    Epsilon = predicted - A
    Epsilon = Epsilon
    delta = np.zeros(A.shape[0])
    delta = delta
    delta[np.sum(predicted, 1) < 0.5] = -1.
    Epsilon[delta < -0.5, :] = 0.
    Epsilon[:, delta < -0.5] = 0.

    return delta, Epsilon, state


def gen_pred(model, A, X, state=None):
    delta_pred, Epsilon_pred = model(torch.tensor(A, dtype=torch.float), torch.tensor(X, dtype=torch.float))
    delta_pred = delta_pred.detach().numpy()
    Epsilon_pred = Epsilon_pred.detach().numpy()
    delta = np.zeros(A.shape[0])
    delta[delta_pred > 0.5] = 1.
    delta[delta_pred < -0.5] = -1.
    Epsilon = np.zeros(A.shape)
    Epsilon[np.logical_and(A > 0.5, Epsilon_pred < -0.5)] = -1.
    Epsilon[np.logical_and(A < 0.5, Epsilon_pred > +0.5)] = +1.
    return delta, Epsilon, state


In [6]:
# EVALUATION FUNCTIONS
eval_criteria = ['node_ins_recall',
                 'node_ins_precision',
                 'node_del_recall',
                 'node_del_precision',
                 'edge_ins_recall',
                 'edge_ins_precision',
                 'edge_del_recall',
                 'edge_del_precision']
# set up a function to compute precision and recall
def prec_rec(X, Y):
    # X is the prediction, Y is the target
    target_insertions = Y > 0.5
    predicted_insertions = X > 0.5
    target_deletions = Y < -0.5
    predicted_deletions = X < -0.5
    # first, check the insertion recall
    if np.sum(target_insertions) < 0.5:
        ins_rec = 1.
    else:
        ins_rec  = np.mean(X[target_insertions] > 0.5)
    # then the insertion precision
    if np.sum(predicted_insertions) < 0.5:
        ins_prec = 1.
    else:
        ins_prec = np.mean(Y[predicted_insertions] > 0.5)
    # then the deletion recall
    if np.sum(target_deletions) < 0.5:
        del_rec = 1.
    else:
        del_rec  = np.mean(X[target_deletions] < -0.5)
    # and finally the deletion precision
    if np.sum(predicted_deletions) < 0.5:
        del_prec = 1.
    else:
        del_prec = np.mean(Y[predicted_deletions] < -0.5)
    return ins_rec, ins_prec, del_rec, del_prec

## Dataset setup

In [7]:
import graph_edit_cycles
import degree_rules
import game_of_life
import random


# DATASET SETUP
def generate_edit_cycle():
    As, Xs, tuples = graph_edit_cycles.generate_time_series(random.randrange(3), random.randrange(12), random.randrange(4, 12))
    deltas = []
    Epsilons = []
    for tpl in tuples:
        deltas.append(tpl[0])
        Epsilons.append(tpl[1])
    return As, Xs, deltas, Epsilons


def generate_degree_rules():
    # the initial number of nodes in each graph
    n_init = 8
    # the maximum number of nodes that can occur in each graph during evolution
    n_max  = n_init * 4
    return degree_rules.generate_time_series_from_random_matrix(n_init, n_max = n_max)


def generate_game_of_life():
    # set hyper-parameters for the game of life random grid generation
    grid_size = 10
    num_shapes = 1
    p = 0.1
    T_max = 10
    A, Xs, deltas = game_of_life.generate_random_time_series(grid_size, num_shapes, p, T_max)
    As = [A] * len(Xs)
    Epsilons = [np.zeros_like(A)] * len(Xs)
    return As, Xs, deltas, Epsilons

In [8]:
# CONFIG FOR EXPERIMENTS

# Models
models = ['VGAE', 'VGRNN', 'GEN_crossent', 'GEN']
setup_funs = [setup_vgae, setup_vgrnn, setup_gen, setup_gen]
loss_funs = [vgae_loss, vgrnn_loss, gen_loss_crossent, gen_loss]
pred_funs = [vgae_pred, vgrnn_pred, gen_pred, gen_pred]

# Datasets
#datasets = ['edit_cycles', 'degree_rules', 'game_of_life']
datasets = ['game_of_life', 'degree_rules']
#dim_ins  = [4, 32, 1]
#dim_ins = [32, 1]
dim_ins = [1, 32]
#generator_funs = [generate_edit_cycle, generate_degree_rules, generate_game_of_life]
generator_funs = [generate_game_of_life, generate_degree_rules]

### Actual Experiment

In [9]:
# iterate over all datasets
for d in range(len(datasets)):
    print('\n--- data set %s ---\n' % datasets[d])
    # load partial runtime results if possible
    runtimes_file = 'results/%s_runtimes.csv' % datasets[d]
    if os.path.exists(runtimes_file):
        runtimes = np.loadtxt(runtimes_file, skiprows = 1, delimiter = '\t')
    else:
        runtimes = np.full((R, len(models)), np.nan)
    # iterate over all models
    for k in range(len(models)):
        print('--- model %s ---' % models[k])
        # load partial results if possible
        results_file = 'results/%s_%s_results.csv' % (datasets[d], models[k])
        curves_file  = 'results/%s_%s_learning_curves.csv' % (datasets[d], models[k])
        if os.path.exists(results_file):
            results = np.loadtxt(results_file, skiprows = 1, delimiter = '\t')
            learning_curves = np.loadtxt(curves_file, delimiter = '\t')
        else:
            results = np.full((R, len(eval_criteria)), np.nan)
            learning_curves = np.full((max_epochs, R), np.nan)
        # iterate over experimental repeats
        for r in range(R):
            # check if this repeat is already evaluated; if so, skip it
            if not np.isnan(learning_curves[0, r]):
                continue
            print('-- repeat %d of %d --' % (r+1, R))
            start_time = time.time()
            # set up model
            if datasets[d] == 'game_of_life':
                nonlin = torch.nn.Sigmoid()
            else:
                nonlin = torch.nn.ReLU()
            model = setup_funs[k](dim_ins[d], nonlin)
            # set up optimizer
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
            # initialize moving loss average for printing
            loss_avg = None
            # start training
            for epoch in range(max_epochs):
                optimizer.zero_grad()
                # sample a time series from the data set
                As, Xs, deltas, Epsilons = generator_funs[d]()
                # compute the loss over all time steps
                loss = 0.
                state = None
                for t in range(len(As)):
                    # compute loss
                    loss_obj, state = loss_funs[k](model, As[t], Xs[t], deltas[t], Epsilons[t], state=state)
                    # compute gradient
                    loss_obj.backward()
                    # accumulate loss
                    loss += loss_obj.item()
                # perform an optimizer step
                optimizer.step()
                # store the current loss value in the learning curve
                learning_curves[epoch, r] = loss
                # compute a new moving average over the loss
                if loss_avg is None:
                    loss_avg = loss
                else:
                    loss_avg = loss_avg * 0.9 + 0.1 * loss
                # print every print_step steps
                if(epoch+1) % print_step == 0:
                    print('loss avg after %d epochs: %g' % (epoch+1, loss_avg))
                # stop early if the moving average is small
                if loss_avg < loss_threshold:
                    break
            # perform evaluation on new time series
            results[r, :] = 0.
            T = 0
            for j in range(N_test):
                # get a random time series from the dataset
                As, Xs, deltas, Epsilons = generator_funs[d]()
                state = None
                for t in range(len(As)):
                    # predict the current time step with the network
                    delta, Epsilon, state = pred_funs[k](model, As[t], Xs[t], state=state)
                    # assess node edit precision and recall
                    results[r, :4] += prec_rec(delta, deltas[t])
                    # assess edge edit precision and recall
                    results[r, 4:] += prec_rec(Epsilon, Epsilons[t])
                    # TODO this is only for debugging purposes
                    if (d != 1 and k == 2 and np.any(results[r, :] < 0.99)) or np.any(np.isnan(results[r, :])):
                        print('delta (predicted) = %s' % str(delta))
                        print('delta (target) = %s' % str(deltas[t]))
                        print('Epsilon (predicted) = %s' % str(Epsilon))
                        print('Epsilon (target) = %s' % str(Epsilons[t]))
                        raise ValueError('stop')
                        
                T += len(As)
            results[r, :] /= T
            # store runtime
            runtimes[r, k] = time.time() - start_time
            np.savetxt(runtimes_file, runtimes, delimiter = '\t', fmt = '%g', header = '\t'.join(models), comments = '')
            # store results
            np.savetxt(results_file, results, delimiter = '\t', fmt = '%g', header = '\t'.join(eval_criteria), comments = '')
            # store learning curves
            np.savetxt(curves_file, learning_curves, delimiter = '\t', fmt = '%g')
        # print results
        for crit in range(len(eval_criteria)):
            print('%s: %g +- %g' % (eval_criteria[crit], np.mean(results[:, crit]), np.std(results[:, crit])))


--- data set game_of_life ---

--- model VGAE ---
node_ins_recall: 0.274 +- 0.0941488
node_ins_precision: 1 +- 0
node_del_recall: 1 +- 0
node_del_precision: 0.0375 +- 0.00260077
edge_ins_recall: 1 +- 0
edge_ins_precision: 1 +- 0
edge_del_recall: 1 +- 0
edge_del_precision: 1 +- 0
--- model VGRNN ---
-- repeat 1 of 5 --
loss avg after 100 epochs: 7.44709
loss avg after 200 epochs: 7.40946
loss avg after 300 epochs: 7.31923
loss avg after 400 epochs: 7.29466
loss avg after 500 epochs: 7.27402
loss avg after 600 epochs: 7.22272
loss avg after 700 epochs: 7.17407
loss avg after 800 epochs: 7.17703
loss avg after 900 epochs: 7.10687
loss avg after 1000 epochs: 7.16115
loss avg after 1100 epochs: 7.14281
loss avg after 1200 epochs: 7.00304
loss avg after 1300 epochs: 7.12211
loss avg after 1400 epochs: 7.1911
loss avg after 1500 epochs: 7.11026
loss avg after 1600 epochs: 7.10524
loss avg after 1700 epochs: 7.16129
loss avg after 1800 epochs: 7.07459
loss avg after 1900 epochs: 7.18062
loss 

RuntimeError: expected scalar type Float but found Double

In [ ]:
# visualize learning curves
import matplotlib.pyplot as plt
smoothing_steps = 10
fig, axes = plt.subplots(ncols=1, nrows=len(datasets))
for d in range(len(datasets)):
    for k in range(len(models)):
        curves_file  = 'results/%s_%s_learning_curves.csv' % (datasets[d], models[k])
        learning_curves = np.loadtxt(curves_file, delimiter = '\t')
        acum = np.cumsum(np.nanmean(learning_curves, 1))
        axes[d].semilogy((acum[smoothing_steps:] - acum[:-smoothing_steps])/smoothing_steps)
    axes[d].set_xlabel('epoch')
    axes[d].set_ylabel('loss')
    axes[d].set_title(datasets[d])
    axes[d].legend(models)
plt.show()